In [4]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import MultiHeadWaveletModel, LEMONDataModule, Julia2018DataModule
from src.acnets.deep import MultiHeadCausalModel


lemon_datamodule = LEMONDataModule(
    atlas='aal', kind='partial correlation',
    test_ratio=.1, val_ratio=.05,
    n_subjects=203, batch_size=16)
lemon_datamodule.setup()

julia2018_datamodule = Julia2018DataModule(
    atlas='aal', kind='partial correlation',
    test_ratio=.25, batch_size=32)
julia2018_datamodule.setup()

n_regions = lemon_datamodule.train[0][0].shape[1]
n_wavelets = 124
n_embeddings = 8
segment_length = 32

model = MultiHeadCausalModel(n_regions, n_embeddings, segment_length)

trainer = model.fit(lemon_datamodule, max_epochs=1000, phase='pretrain')
tuner = model.fit(julia2018_datamodule, max_epochs=5000, phase='finetune')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /home/morteza/workspace/ACNets/models/checkpoints/causal exists and is not empty.
Restoring states from the checkpoint path at /home/morteza/workspace/ACNets/models/checkpoints/causal/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name              ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy          │ MulticlassAccuracy │      0 │
│ 1 │ feature_extractor │ CausalVAE          │ 11.8 K │
│ 2 │ cls_head          │ Classifier         │    118 │
└───┴───────────────────┴────────────────────┴────────┘

Trainable params: 11.9 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 11.9 K                                                                                               
Total estimated model params size (MB): 0

Restored all states from the checkpoint at /home/morteza/workspace/ACNets/models/checkpoints/causal/last.ckpt


Output()

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name              ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy          │ MulticlassAccuracy │      0 │
│ 1 │ feature_extractor │ CausalVAE          │ 11.8 K │
│ 2 │ cls_head          │ Classifier         │    118 │
└───┴───────────────────┴────────────────────┴────────┘

Trainable params: 0                                                                                                
Non-trainable params: 11.9 K                                                                                       
Total params: 11.9 K                                                                                               
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=5000` reached.
